<h1> Generate Product Description Using Amazon Bedrock </h1>
<br>

<p> Before starting, please make sure this notebook is using <b>conda_python3</b> kernel from the top right! </p>

<p> In this notebook, let's learn how to generate product description using Amazon Bedrock and <a href="https://python.langchain.com/docs/get_started/introduction" target="_blank">Langchain</a>. </p>

<p> To learn more about Amazon Bedrock integration with LangChain, please make a note of this <a href="https://python.langchain.com/docs/integrations/llms/bedrock" target="_blank">documentation</a> to take a look at <b>after</b> the completion of this workshop. </p>

<p> To run this notebook, go to Cell -> Run All. Inspect the output of each cell block. </p>

<b> Please read the following instructions carefully! </b>
    <ul>
    <li>We highly recommend you to run all cells and inspect output rather than running the cells individually to save time as well as avoid any issues.  
    <li>This notebook is for your understanding. Running this notebook is NOT required for proceeding with the next steps of your workshop.
    <li>In case your notebook does not run as expected or if you run into any errors, please proceed with the next steps provided in the Workshop instructions. 
    <li>If you choose to run the notebooks, please read the comments in the markdown and inspect the output of each cell.
    </ul>

<h3> Install required dependencies </h3>
<p> <b>Note:</b> If you notice any ERRORs from the following cell, ignore them and proceed with the next cells. This command may take 1-2 mins to run.</p><br>

In [ ]:
%pip install --quiet --no-build-isolation --force-reinstall \
    "boto3==1.28.63" \
    "awscli==1.29.63" \
    "botocore==1.31.63" \
    "langchain==0.0.309"

<h3> Import required packages </h3>

In [ ]:
import json
import os
import sys
import boto3
import botocore
from langchain import PromptTemplate
from langchain.llms.bedrock import Bedrock

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

<h3> Initialize Bedrock client </h3><br>

In [ ]:
 boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

<h3>Set inference parameters for LLM</h3>
<p> Setting these values are optional. The parameters change based on the LLM used. In this case, we are going to use Claude Anthropic LLM. For more details, refer to this <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html" target="_blank">documentation</a>. </p><br>

In [ ]:
inference_modifier = {}
inference_modifier['max_tokens_to_sample'] = 200
inference_modifier['temperature'] = 0.5
inference_modifier['top_k'] = 250
inference_modifier['top_p'] = 1
inference_modifier['stop_sequences'] = ["\n\nHuman"]

<h3>Provide details about a product</h3>
<p> This data will be used to construct our prompt which will be passed to the LLM to generate product description. </p> 

In [ ]:
product_brand = "Legendaire"
product_category = "Shirt"
product_name = "Legandaire Shirt"

# Brief data about the product
product_details = """
        - collared white shirt 
        - 80% cotton 20% polyester
        - semi-casual
        - great for office or golfing
        - comfortable breathable material
        - flex fit
    """

# Colors available for the product 
product_colors = ["White", "Black", "Blue"]

# Length of the desired product description (generated from LLM)
max_length = 100

<h3> Initialize LLM </h3>
<p> Using Langchain, initialize an LLM instance for text generation. </p>

In [ ]:
# Passing Anthropic Claude as model ID

textgen_llm = Bedrock(
            model_id="anthropic.claude-instant-v1",
            client=boto3_bedrock,
            model_kwargs=inference_modifier,
        )

<p> Create a prompt template with variables like product details, category, brand, available colors etc. Read the template parameter carefully to understand the prompting instructions. </p> 

In [ ]:
prompt_template = PromptTemplate(
            input_variables=["brand", "colors", "category", "length", "name","details"], 
            template="""
            
                Human: Create a catchy product description for a {category} from the brand {brand}. 
                Product name is {name}. 
                The number of words should be less than {length}. 

                Following are the product details:  

                <product_details>
                {details}
                </product_details>

                Briefly mention about all the available colors of the product.

                Example: Available colors are Blue, Purple and Orange. 

                If the <available_colors> is empty, don't mention anything about the color of the product.

                <available_colors>
                {colors}
                </available_colors>

                Assistant:
                
                """
    
            )

<p> Let's construct the prompt by passing all the variables to the prompt template. </p>

In [ ]:
 # Pass in form values to the prompt template

prompt = prompt_template.format(brand=product_brand, 
                                colors=product_colors,
                                category=product_category,
                                length=max_length,
                                name=product_name,
                                details=product_details)

<h3> Call Bedrock with the constructed prompt </h3>

In [ ]:
response = textgen_llm(prompt)
print_ww(response)

<h4> Get the second paragraph i.e, only the product description </h4>

In [ ]:
generated_response = response[response.index('\n')+1:]
print_ww(generated_response)

<h4> We can also try to generate the product descripton using another LLM. Let's use Amazon Titan LLM from Bedrock </h4>

In [ ]:
# Initialize Amazon Titan LLM from Bedrock
textgen_llm = Bedrock(
                model_id="amazon.titan-tg1-large",
                client=boto3_bedrock)

# Call the LLM with same prompt
response = textgen_llm(prompt)
print_ww(response)

<h3> You've successfully generated description for a product with Anthropic Claude LLM!</h3>

<p> Please stop the notebook kernel before proceeding. </p>

<h4> Now, let's learn how to integrate Amazon Bedrock and Langchain into your web application to do the same. Please go back to Workshop Studio and follow the instructions to replicate this code into your Cloud9 environment. </h4>